In [1]:
from IPython.display import display
from numpy.random import RandomState
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pywt
import numpy as np
import pandas as pd
from scipy.fftpack import fft, dct

In [2]:
def first_step(image):
    l,m =image.shape
    u1 = np.array(np.sum(image,axis=1)).reshape(l,1)
    u =np.array(u1/np.linalg.norm(u1))
    v1 = np.array(np.sum(image,axis=0)).reshape(m,1)
    v =np.array(v1/np.linalg.norm(v1))
    return l,m,u,v

def decomposition(x_batch,limit):
    l,m,u_1,v_1=first_step(x_batch)
    eigen = []
    u=u_1
    v=v_1
    A=x_batch
    for i in range(limit):
        j,k =A.shape
        alfa = np.matmul(np.matmul(u_1.transpose(),A),v_1)
        bet1 = np.eye(j)-(np.matmul(u_1,u_1.transpose()))
        bet2 = np.matmul(np.matmul(bet1,A),v_1)
        beta = np.linalg.norm(bet2)
        u_n = np.array(bet2/beta)
        gam1 =np.eye(k)-(np.matmul(v_1,v_1.transpose()))
        gam2 =np.matmul(np.matmul(gam1,A.transpose()),u_1)
        gama = np.linalg.norm(gam2)
        v_n =np.array(gam2/gama)
        A_n =np.matmul(np.matmul(bet1,A),gam1)
        S_n =alfa*u_1*v_1.transpose()+beta*u_n*v_1.transpose()+gama*u_1*v_n.transpose()
 
        eigen.append(alfa[0][0])
        eigen.append(beta)
        eigen.append(gama)
        
        u_1=u_n
        v_1=v_n
        A=A_n
        
    return np.array(eigen)

In [3]:
X = pd.read_csv('EEG.csv', sep=','  , engine='python')
X.drop("Unnamed: 0", axis=1, inplace=True,)

In [4]:
X["X179"]=0
X["X180"]=0

In [5]:
X['y'] = X['y'].replace([2,3, 4, 5], 0 )

In [6]:
y = X["y"].copy()
np.savetxt("eeg_label.csv", y, delimiter=",")
X.drop("y", axis=1, inplace=True)

In [7]:
X_list=X.values.tolist()

In [8]:
tmempr_list = []
wavelet_list = []
dct_list = []
for i in range(11500):
    A=decomposition(np.array(X_list[i]).reshape((15,12)),10)
    tmempr_list.append(A)
    cA, cD = pywt.dwt(X_list[i],'haar')
    cA1, cD1 = pywt.dwt(cA,'haar')
    cA2, cD2 = pywt.dwt(cA1,'haar')
    cA2 = cA2[:30]
    wavelet_list.append(cA2)
    
    a=dct(np.array(X_list[i]).reshape((15,12)))
    solution=[[] for i in range(6+6-1)]
    for k in range(6): 
        for m in range(6): 
            sum=k+m 
            if(sum%2 ==0): 
  
                #add at beginning 
                solution[sum].insert(0,a[k][m]) 
            else: 
                #add at end of the list 
                solution[sum].append(a[k][m])
    b=[]
    for z in range(len(solution)):
        for y in range(len(solution[z])):
                        b.append(solution[z][y])
    b = b[:30]
    
    dct_list.append(b)
    
np.savetxt("eeg_tmempr.csv", tmempr_list, delimiter=",")
np.savetxt("eeg_wavelet.csv", wavelet_list, delimiter=",")
np.savetxt("eeg_dct.csv", dct_list, delimiter=",")